In [1]:
!pip install cassandra-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 55.7 MB/s eta 0:00:00


In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config = {
    "secure_connect_bundle": "secure-connect-nyc-yellow-taxi-db.zip"  # replace <</PATH/TO/>> with the path where your downloaded bundle was downloaded (make sure to place this python file in the same place as well)
}
auth_provider = PlainTextAuthProvider(
    "yiGvbsUpoEqllXBsXXwOjZBc",
    "p,b-6bj-cMT1IR3dv,kGm6A8Q+yZg0jcMHWivO2XU0y9+evSLpYZ923mLvs,ocJ-CLeOdYI8EeiBoa7jR.r3fOFtpiQwO+Zdb2aq4FqaO4xSEXj_TF4KFkHoAs5N6mt_",
)  # replace <<CLIENT ID>> and <<CLIENT SECRET>> with the ClientID and Client Secret from your generated token
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace(
    "nyc_yellow_taxi_db")  # replace <<KEYSPACENAME>> with the name of the keyspace you created in DataStax

In [27]:
# -✔️- Reading listings from CSV into dataframe

import pandas as pd

# read the csv files into the dataframes
trip_data = pd.read_csv('taxi_trip_data.csv')
zone_data = pd.read_csv('taxi_zone_geo.csv')

In [28]:
# a) Remove the columns “store_and_fwd_flag”, “rate_code” and “total_amount” from taxitripdata
trip_data = trip_data.drop(['store_and_fwd_flag', 'rate_code', 'total_amount'], axis=1)


In [29]:
# b) Drop rows with missing details
trip_data = trip_data.dropna()


In [30]:
trip_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 14 columns):
 #   Column               Dtype  
---  ------               -----  
 0   vendor_id            int64  
 1   pickup_datetime      object 
 2   dropoff_datetime     object 
 3   passenger_count      int64  
 4   trip_distance        float64
 5   payment_type         int64  
 6   fare_amount          float64
 7   extra                float64
 8   mta_tax              float64
 9   tip_amount           float64
 10  tolls_amount         float64
 11  imp_surcharge        float64
 12  pickup_location_id   int64  
 13  dropoff_location_id  int64  
dtypes: float64(7), int64(5), object(2)
memory usage: 1.0+ GB


In [31]:
zone_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   zone_id    263 non-null    int64 
 1   zone_name  263 non-null    object
 2   borough    263 non-null    object
 3   zone_geom  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.3+ KB


In [32]:
# columns in the trip_data table:
# vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, rate_code, store_and_fwd_flag, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount,
# imp_surcharge, total_amount, pickup_location_id, dropoff_location_id, pickup_location, dropoff_location
#  zone_id_x zone_name_x  borough_x  zone_geom_x  zone_id_y zone_name_y  borough_y  zone_geom_y

session.execute(
    """
    DROP TABLE trip_data;
    """
)

# create the table for trip_data
session.execute(
    """
CREATE TABLE trip_data (
    trip_id UUID PRIMARY KEY,
    vendor_id INT,
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    passenger_count FLOAT,
    trip_distance FLOAT,
    payment_type INT,
    fare_amount FLOAT,
    extra FLOAT,
    mta_tax FLOAT,
    tip_amount FLOAT,
    tolls_amount FLOAT,
    imp_surcharge FLOAT,
    pickup_location INT,
    dropoff_location INT,
);
    """
)

In [39]:
# c) Insert the data in the database as you see fit

# Insert the data into the trip_data table
# instert only 20,000 rows

for index, row in trip_data.head(20000).iterrows():

    # Extract data for insertion
    vendor_id = row['vendor_id']
    pickup_datetime = row['pickup_datetime']
    dropoff_datetime = row['dropoff_datetime']
    passenger_count = row['passenger_count']
    trip_distance = row['trip_distance']
    payment_type = row['payment_type']
    fare_amount = row['fare_amount']
    extra = row['extra']
    mta_tax = row['mta_tax']
    tip_amount = row['tip_amount']
    tolls_amount = row['tolls_amount']
    imp_surcharge = row['imp_surcharge']
    pickup_location = row['pickup_location_id']
    dropoff_location = row['dropoff_location_id']

    # Insert the data into the trip_data table
    session.execute(
        """
        INSERT INTO trip_data (trip_id,vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, imp_surcharge, pickup_location, dropoff_location)
        VALUES (UUID(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (
            vendor_id,
            pickup_datetime,
            dropoff_datetime,
            passenger_count,
            trip_distance,
            payment_type,
            fare_amount,
            extra,
            mta_tax,
            tip_amount,
            tolls_amount,
            imp_surcharge,
            pickup_location,
            dropoff_location,
        ),
    )

In [42]:
# d) Calculate the duration for each trip and add it as a new field in your database (e7sb el duration men pickup date le dropoff date)
# add a new column to the trip_data dataframe for the duration of the trip


KeyError: 'trip_id'

In [ ]:
# e) Use “fare_amount”, “extra”, “mta_tax”, “tip_amount”, “tolls_amount” and “imp_surcharge” to calculate the total trip cost and add it as a new field in your database (e7sb el total cost)

In [ ]:
# f) What is the most common payment type used per time of day? Hint: time of day meaning morning, afternoon, evening, night
# create a new column in the trip_data dataframe for the time of day


In [ ]:
# g) What is the average tip amount per passenger count?
# query the trip_data table to find the average tip amount per passenger count


In [41]:
# h) What are the best 5 locations for drivers to pick up passengers from? (based on the trip count)
# query the trip_data table to find the best 5 locations for drivers to pick up passengers from


SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 4:25 no viable alternative at input '.' (...(*)    FROM trip_data    WHERE [pickup_location]....)">

In [ ]:
# Is there a correlation between trip distance and the tip amount? (not to be done using the correlation calculation)
# create a new cloumn tip percentage, and calculate the tip percentage for each trip by dividing the tip amount by the total amount, and compare it to the trip distance

In [ ]:
# Display the results of f, g and h using visualizations (will be graded based on the creativity and efficiency of the visualizations)
